# オリジナル画像認識モデル生成チュートリアル(YoloV7版)

このチュートリアルでは、オリジナルのアノテーション済み画像を使って、物体検出モデルの学習を行います。  
下記をベースに作成されています。  
https://github.com/luxonis/depthai-ml-training/blob/master/colab-notebooks/YoloV7_training.ipynb



# 1. 初期設定  
上部メニューからランタイム→「ランタイムのタイプを変更」→「GPU」を選択します。



# 2. 環境構築

## 2.1 YoloV7のクローン

YoloV7をクローンします。

In [ ]:
%cd /content/
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7/

## 2.2 YoloV7のdependencyのインストール
YoloV7を動かすのに必要なdependencyのインストールを行います。

In [ ]:
!pip install -U -r requirements.txt  # install dependencies

#3. データセットの準備

## 3.1 Google Driveのマウント
Colab上にGoogle Driveをマウントします。  
Google Driveの接続許可を求めるウィンドウが開くので、許可して接続してください。  
Google Driveの中身は`/content/gdrive`に表示されます。

In [ ]:
#mount your google drive.
#it will be visible in the file navigator on the left of this notebook
#there should be a folder in your drive with your data
from google.colab import drive
drive.mount('/content/gdrive')

## 3.2 データフォルダの指定
下記トレーニングフォルダ、バリデーションフォルダ、テストフォルダーの3箇所のパスを指定します。
**下記のフォルダ名を自身のGoogle Driveのパスに合わせて変更して下さい。**  
**画像データ(.jpg)とアノテーションデータ(YOLO形式の.txt)をトレーニングフォルダに入れます**。  
**classes.txtを親ディレクトリに入れます。**  
**テストフォルダには最終テスト用の画像を数枚入れます。こちらはアノテーションの必要はありません。**


In [ ]:
#copy files from gdrive to colab drive. this takes a few minutes, depending on the number of files.
#Go on the file explorer on the left of this notebook and access your gdrive. find the folders
#with your train, test and final_test images.
#Right click on each and copy the path. paste it btw the first " " in the corresponding lines  
import os
!mkdir "/content/yolov7/dataset/"
# GoogleDriveのyolov7 datasetのフォルダ。
YOUR_GDRIVE_PATH ="/content/gdrive/MyDrive/object_detection"
#トレーニングフォルダ。左のパスを自身のGoogle Driveのトレーニングフォルダのパスに変更
TRAIN_DIR = YOUR_GDRIVE_PATH +"/images/train"
#テストフォルダ。左のパスを自身のGoogle Driveのトレーニングフォルダのパスに変更
TEST_DIR = YOUR_GDRIVE_PATH + "/images/test"
# クラスファイルのパス
CLASS_FILE = TRAIN_DIR + "/classes.txt"
##出力フォルダのパス
OUTPUT_DIR = YOUR_GDRIVE_PATH + "/output"
!mkdir {OUTPUT_DIR}

##学習で使うデータフォルダ。
DATA_DIR = "/content/yolov7/dataset"
!mkdir {DATA_DIR}
!cp -r {TEST_DIR} {DATA_DIR}

## 3.3 トレーニングデータの振り分け
トレーニングディレクトリのファイルの8割を学習に、2割をバリデーション用に振り分けます。

In [ ]:
import glob
import shutil
import random

# 学習データの割合(Percentage of training data)
train_ratio = 0.8

train_directory = DATA_DIR + "/train"
%rm -rf {train_directory}
train_images_directory = train_directory + "/images"
train_labels_directory = train_directory + "/labels"
valid_directory = DATA_DIR + "/valid"
%rm -rf {valid_directory}
valid_images_directory = valid_directory + "/images"
valid_labels_directory = valid_directory + "/labels"
!mkdir {train_directory}
!mkdir {train_directory}/images
!mkdir {train_directory}/labels
!mkdir {valid_directory}
!mkdir {valid_directory}/images
!mkdir {valid_directory}/labels
test_directory = DATA_DIR + "/test"

# コピー元ファイルリスト取得(Get copy source file list)
annotation_list = sorted(glob.glob(TRAIN_DIR + '/*.txt'))
image_list = sorted(glob.glob(TRAIN_DIR + '/*.jpg'))

file_num = len(annotation_list)
# インデックスシャッフル(shuffle)
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(index_list):
    if count < int(file_num * train_ratio):
        # 学習用データ(Training Data)
        shutil.copy2(annotation_list[index], train_labels_directory)
        shutil.copy2(image_list[index], train_images_directory)
    else:
        # 検証用データ(Validation Data)
        shutil.copy2(annotation_list[index], valid_labels_directory)
        shutil.copy2(image_list[index], valid_images_directory)

## 3.4 ファイルチェック
下記を実行すると、トレーニングフォルダ内のファイル一覧が表示されます。
トレーニングフォルダに入れていた画像の8割が正しくコピーされているか確認してください。

In [ ]:
# quick check for training data files. you can also browse to the object_detection_demo_flows
# on the left and see if they were copied
!ls {train_images_directory}

# 4.学習の準備

## 4.1 学習用の設定ファイルの作成

In [ ]:
import yaml

f = open(CLASS_FILE)
class_list = f.read().split("\n")
class_list = [a for a in class_list if a != '']
data = {}
data["train"] = DATA_DIR + "/train"
data["val"] = DATA_DIR + "/valid"
data["test"] = DATA_DIR + "/test"

data["nc"] = len(class_list)
data["names"] = class_list
print(data)
with open('./data/yolov7_dataset.yaml', 'w') as file:
    yaml.dump(data, file)

## 4.2 学習済みモデルの重みの取得

今回はYolov7-tinyを使用します。

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

## 4.3 コンフィグファイルの作成
[`yolov7/cfg/training/yolov7-tiny.yaml`](https://github.com/WongKinYiu/yolov7/blob/main/cfg/training/yolov7-tiny.yaml)をベースにコンフィグファイルを作成します。  
classes.txtを元に、クラス数(=nc)のみ書き換えを行います。

In [ ]:
import yaml

with open('./data/yolov7_dataset.yaml') as file:
    obj = yaml.safe_load(file)
    nc = obj["nc"]
  
with open('/content/yolov7/cfg/training/yolov7-tiny.yaml') as param_file:
    param = yaml.safe_load(param_file)
    param["nc"] = nc

with open('/content/yolov7/cfg/training/yolov7-tiny-updated.yaml', 'w') as file:
    yaml.dump(param, file, default_flow_style=True)

#5.学習の実行

##5.1 学習の実行
学習を行います。  
--epochsの値を変えることで、学習回数を変更することができます。増やすほど学習に時間がかかります。

In [ ]:
%rm -rf /content/yolov7/runs/train/result/*
%rm -rf /content/yolov7/runs/test/result/*
!python3 train.py --epochs 300 --workers 8 --device 0 --batch-size 16 --data data/yolov7_dataset.yaml --img 640 640 --cfg /content/yolov7/cfg/training/yolov7-tiny-updated.yaml --weights 'yolov7-tiny.pt' --hyp data/hyp.scratch.tiny.yaml --name result --exist-ok

# 6.バリデーション結果の確認

## 6.1 validation結果のグラフを表示する
validation結果のplotを表示します。  
Box：Bounding Boxの予測がオブジェクトを正確にカバーしないことによる損失。  
Objectness：ボックスとオブジェクトのIoUの予測が間違っていたことによる損失。  
Classification：そのボックス内のオブジェクトについて、正しいクラスは'1'、他のすべてのクラスは'0'と予測することから逸脱することによる損失。  
Precision: 適合率。検出結果の中に、適合しないものが入っていない割合  
P = tp / (tp + fp)  
Recall: 再現率。すべてのデータのうち、漏れなく検出できた割合
R = tp / (tp + fn)  
mAP = Mean Average Precision、平均適合率の平均。学習が収束しているかどうかの目安にする。  


In [ ]:
import glob
from IPython.display import Image
from IPython.display import display_png

display_png(Image('/content/yolov7/runs/train/result/results.png'))


#7. テストの実行

#7.1 テストの実行
学習させたモデルを用いてtestディレクトリに入れたファイルを用いてテストを行い、結果を表示します。

In [ ]:
!python test.py --data data/yolov7_dataset.yaml --iou-thres 0.65 --device 0 --weights runs/train/result/weights/best.pt --task test --name result --exist-ok

## 7.2 テスト画像による推論の実行
学習させたモデルに対して、testディレクトリに入れたファイルを用いて推論を実施し、結果画像を出力します。

In [ ]:
import glob
from IPython.display import Image
from IPython.display import display_jpeg

print(image_list)
!python detect.py --weights runs/train/result/weights/best.pt --img-size 640 --source 'dataset/test' --name result --exist-ok

results_path = sorted(glob.glob("runs/detect/result/*.jpg"))
for image_path in results_path:
  print(image_path)
  display_jpeg(Image(image_path))

# 8.モデルのダウンロード

## 8.1 モデルのダウンロード
学習結果のpytorchモデルをダウンロードします。  
実行すると、ダウンロードウインドウが開くので、任意の場所にダウンロードしてください。

In [ ]:
from google.colab import files
import os

!cp "/content/yolov7/runs/train/result/weights/best.pt" {OUTPUT_DIR}/yolov7.pt
files.download(OUTPUT_DIR + "/yolov7.pt") 

以上で学習は終了です。  
マニュアルに戻り、次の手順に進んでください。